In [34]:
import random
import numpy as np
#Leduc Hold'em is a simplified poker game in which each player gets 1 card
#Each player automatically puts 1 chip into the pot to begin the hand
#This is followed by the first round (called preflop) of betting
#In this round the bet size is 2 and there is a maximum of 2 bets
#If neither player folds, the hand goes on to the second round (the flop)
#In this round the bet size is 4 and again there is a maximum of 2 bets
#If neither player folds, the pot is given to the best hand
#The best hand is either a pair if the community card matches a player's private card
#Or simply the highest private card if neither player matches the community card

def validbets(r1history,r2history,rnd):
    history = [r1history, r2history]
    if rnd == 1:
        bet = 2
    else:
        bet = 4
    
    f,k,c,b,r = True, True, True, True, True #default all to valid
    betoptions = [f,k,c,b,r] #fold check call bet raise
    
    if history[rnd-1][-1:] == 'b' or history[rnd-1][-1:] == 'r':
        betoptions[3] = False #Bet invalid -- Can only bet at the start of a round or after a check
        betoptions[1] = False #Check invalid -- Can only check at the start of a round or after a check
        
    if history[rnd-1][-1:] != 'b':
        betoptions[4] = False #Raise invalid -- Can only raise after a bet (and only one raise per round)
        
    if history[rnd-1][-1:] == 'k' or history[rnd-1][-1:] == '':
        betoptions[0] = False #Fold invalid -- Can only fold after a bet or a raise
        betoptions[2] = False #Call invalid -- Can only call after a bet or a raise
        
    return makebet(betoptions,bet)
    
def makebet(betoptions,bet):
    print('Betting options')
    if betoptions[0]: print('FOLD: f')
    if betoptions[1]: print('CHECK: k')
    if betoptions[2]: print('CALL {}: c'.format(bet))
    if betoptions[3]: print('BET {}: b'.format(bet))
    if betoptions[4]: print('RAISE {}: r'.format(bet*2))

    if betinput in betoptions:
        return betinput
    else:
        print('Invalid bet made, try again')
        betnow(betoptions)

def playpreflop(p0card,p1card,flopcard,r1history,r2history,dealer,profit0):
    #bet size is fixed = 2 for this round
    print(r1history, r2history)
    plays = len(r1history)
    actingplayer = (plays+dealer) % 2 #players are either player0 or player1
    opponentplayer = 1 - actingplayer
    pot = 2+ 2*r1history.count('b')+2*r1history.count('c')+4*r1history.count('r')

    if r1history[-1:] == 'f': #game over, last player folded
        profit = (pot - 2)/2 #subtract uncalled bet of 2 and divide by 2 (since profit is opponent's half)
        if activeplayer == 0:
            profit0 +=profit
        else:
            profit0 -=profit
        print('End of game! Player {} wins pot of {} (profits {})\n'.format(activeplayer,pot,profit))
        main(1-dealer,profit0)
    
    if r1history[-2:] == 'kk' or r1history[-2:] == 'bc' or r1history[-2:] == 'rc': #2 checks in a row or a bet/raise and a call
        print('\nDealing flop card: {}'.format(flopcard))
        playflop(p0card,p1card,flopcard,r1history,r2history,dealer,profit0)
          
    print('\nPot size: {}'.format(pot))       
    print('Player {} turn to act'.format(actingplayer))
    bet = validbets(r1history,r2history,1)
    playpreflop(p0card,p1card,flopcard,r1history+bet,r2history,dealer,profit0)

    
def playflop(p0card,p1card,flopcard,r1history,r2history,dealer,profit0):
    #bet size is fixed = 4 for this round
    print(r1history, r2history)
    plays = len(r2history+r1history)
    opponentplayer = (plays+dealer) % 2 #players are either player0 or player1
    actingplayer = 1 - opponentplayer
    pot = 2+ 2*r1history.count('b')+2*r1history.count('c')+4*r1history.count('r')+4*r2history.count('b')+4*r2history.count('c')+8*r2history.count('r')

    if r2history[-1:] == 'f': #game over, last player folded
        profit = (pot - 4)/2
        if activeplayer == 0:
            profit0 +=profit
        else:
            profit0 -=profit
        print('End of game! Player {} wins pot of {} (profits {})\n'.format(actingplayer,pot,profit))
        main(dealer-1, profit0)
    elif r2history[-2:] == 'kk' or r2history[-2:] == 'bc': #2 checks in a row or a bet and a call   
        if p0card == flopcard:
            winner = 0
            winninghand = [p0card, flopcard]
        elif p1card == flopcard:
            winner = 1
            winninghand = [p1card, flopcard]
        elif p1card > p0card:
            winner = 1
            winninghand = [p1card, flopcard]
        elif p0card > p1card:
            winner = 0
            winninghand = [p0card, flopcard]
        else:
            winninghand = [p0card, flopcard]
            print('Tie game! Pot size {} and both players had {}\n'.format(pot,winninghand))
            main(1-dealer,profit0)
        profit = pot/2
        if winner == 0:
            profit0 +=profit
        else:
            profit0 -=profit
        print('End of game! Player {} wins pot of {} with {} (profits {})\n'.format(winner,pot,winninghand,profit))
        main(1-dealer,profit0)
    
    print('\nPot size: {}'.format(pot))
    print('Player {} turn to act'.format(actingplayer))
    bet = validbets(r1history,r2history,2)
    playflop(p0card,p1card,flopcard,r1history,r2history+bet,dealer,profit0)
    

def main(dealer, profit0):
    #here we shuffle the deck and define a card for
    #each player and the shared community card (only
    #relevant if the players get to that betting round)
    cards = [1,1,2,2,3,3] #define 6 card poker deck
    cards_shuffled = np.random.shuffle(cards) #shuffle
    if dealer == 0: #deal cards according to the standard order
        p0card = cards[1]
        p1card = cards[0]
    else:
        p0card = cards[0]
        p1card = cards[1]
    flopcard = cards[2] #the shared community card is the next card in the deck
    print('=================\nStarting new game\n=================')
    print('Player 0 profit: {}'.format(profit0))
    print('Player 1 profit: {}'.format(-profit0))
    print ('')
    print('Player 0 card: {}'.format(p0card))
    print('Player 1 card: {}'.format(p1card))
    print ('')
    playpreflop(p0card,p1card,flopcard,'','',dealer,profit0) #each player puts 1 into the pot automatically
    
main(0,0) #define dealer as p0 or p1 (dealer is the term for the player who acts first)

Starting new game
Player 0 profit: 0
Player 1 profit: 0

Player 0 card: 1
Player 1 card: 3

 

Pot size: 2
Player 0 turn to act
Betting options
CHECK: k
BET 2: b


NameError: name 'betinput' is not defined

In [ ]:
a = [1, 2, False, 3, False]
b = random.choice(a)
print(b)

In [45]:
a='h'
print(a[-2:])

h


In [2]:
bet, rse, stn = True, True, True
bet

True

In [6]:
if True: print('stone')

stone


In [12]:
fld, chk, cll, bet, rse = True, True, True, True, True
betoptions = [fld,chk,cll,bet,rse]
for i in betoptions:
    i = True
    
print(betoptions)

[True, True, True, True, True]


In [13]:
r1 = 'bd'
r2 = 'cf'
history = [r1, r2]
print(history[0][0])

b


In [18]:
betoptions = [f,k,c,b,r] #fold check call bet raise
for i in betoptions:
    i = True

print(betoptions)

NameError: name 'f' is not defined